In [1]:
!pip install -q https://github.com/buguroo/pyknow/archive/v1.7.0.tar.gz

In [2]:
!pip install yfinance==0.1.80

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import collections
import collections.abc

if not hasattr(collections, 'Mapping'):
    collections.Mapping=collections.abc.Mapping

from pyknow import *
import random

# test


In [4]:
from pyknow import KnowledgeEngine, Rule, MATCH, Fact
import random

class TicTacToeState(Fact):
    pass

class TicTacToe(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def reset(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def display_board(self):
        for row in self.board:
            print(" {} | {} | {} ".format(*row))
            print("---|---|---")

    def make_move(self, player, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        if self.board[row][col] == " ":
            self.board[row][col] = player
            self.remaining_moves.remove(position)
            if self.check_win(player):
                self.game_over = True
                print(f"Player {player} wins!")
                
        else:
            print("Invalid move! Position already taken.")

    def check_win(self, player):
        # Check rows
        for row in self.board:
            if all(cell == player for cell in row):
                return True
        # Check columns
        for col in range(3):
            if all(self.board[row][col] == player for row in range(3)):
                return True
        # Check diagonals
        if all(self.board[i][i] == player for i in range(3)) or all(self.board[i][2 - i] == player for i in range(3)):
            return True
        return False

class TicTacToePlayer(KnowledgeEngine):
    def __init__(self, board):
        super().__init__()
        self.board = board
        self.computer_move = None  # Store the computer's move

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves))
    def get_computer_move(self, remaining_moves):
        valid_moves = [move for move in remaining_moves if self.is_valid(move)]
        position = random.choice(valid_moves)
        self.computer_move = position  # Store the selected position

    def is_valid(self, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        return self.board[row][col] == " "

def play_game():
    game = TicTacToe()
    print("Game Start")
    
    while not game.game_over:
        print("Human's Turn :")
        game.display_board()
        human_move = input("Enter your move (1-9): ")
        
        game.make_move("X", int(human_move))
        print("Human chose position", human_move)
        if game.remaining_moves and not game.game_over:
            # print("---------------")
            computer_player = TicTacToePlayer(game.board)
            computer_player.reset()
            computer_player.declare(TicTacToeState(remaining_moves=game.remaining_moves))
            computer_player.get_computer_move(game.remaining_moves)
            computer_player.run()

            # Access the computer player's facts
            # computer_facts = computer_player.facts
            # print("Computer player facts:", computer_facts)

            if computer_player.computer_move:
                position = computer_player.computer_move
                print(f"Computer chose position {position}.")
                game.make_move("O", position)
                if game.game_over:
                    game.display_board()
                    print("Computer wins!")
            else:
                print("No move made by the computer!")

               #game.make_move("O", int(computer_move))
        elif not game.game_over:
            print("It's a tie!")
            break

    game.display_board()

if __name__ == "__main__":
    play_game()


Game Start
Human's Turn :
   |   |   
---|---|---
   |   |   
---|---|---
   |   |   
---|---|---
Human chose position 4
Computer chose position 9.
Human's Turn :
   |   |   
---|---|---
 X |   |   
---|---|---
   |   | O 
---|---|---
Human chose position 5
Computer chose position 8.
Human's Turn :
   |   |   
---|---|---
 X | X |   
---|---|---
   | O | O 
---|---|---
Player X wins!
Human chose position 6
   |   |   
---|---|---
 X | X | X 
---|---|---
   | O | O 
---|---|---


## advanced test

In [5]:
from pyknow import KnowledgeEngine, Rule, MATCH, Fact
import random

class TicTacToeState(Fact):
    pass

class TicTacToe(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def reset(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def display_board(self):
        for row in self.board:
            print(" {} | {} | {} ".format(*row))
            print("---|---|---")

    def display_board_initial(self):
        count = 1
        for row in self.board:
            for cell in row:
                
                print(" {} ".format(count if cell == " " else cell), end=" ")
                count += 1
            print("\n")

    def make_move(self, player, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        if self.board[row][col] == " ":
            self.board[row][col] = player
            self.remaining_moves.remove(position)
            if self.check_win(player):
                self.game_over = True
                print(f"Player {player} wins!")                
        else:
            print("Invalid move! Position already taken.")
            return False  # Indicate that the move was invalid
        return True  # Indicate that the move was valid


    def check_win(self, player):
        # Check rows
        for row in self.board:
            if all(cell == player for cell in row):
                return True
        # Check columns
        for col in range(3):
            if all(self.board[row][col] == player for row in range(3)):
                return True
        # Check diagonals
        if all(self.board[i][i] == player for i in range(3)) or all(self.board[i][2 - i] == player for i in range(3)):
            return True
        return False

class TicTacToePlayer(KnowledgeEngine):
    def __init__(self, board):
        super().__init__()
        self.board = board
        self.computer_move = None  # Store the computer's move

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves))
    def get_computer_move(self, remaining_moves):
        valid_moves = [move for move in remaining_moves if self.is_valid(move)]
        position = random.choice(valid_moves)
        self.computer_move = position  # Store the selected position

    def is_valid(self, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        return self.board[row][col] == " "

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={2, 4, 6, 8})))
    def prioritize_corner(self):
        if 5 in self.board:
            self.computer_move = 5
        else:
            corners = [1, 3, 7, 9]
            valid_corners = [corner for corner in corners if self.is_valid(corner)]
            if valid_corners:
                self.computer_move = random.choice(valid_corners)

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={5})))
    def choose_center(self):
        if self.is_valid(5):
            self.computer_move = 5

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={1, 3, 7, 9})))
    def prioritize_edges(self):
        edges = [2, 4, 6, 8]
        valid_edges = [edge for edge in edges if self.is_valid(edge)]
        if valid_edges:
            self.computer_move = random.choice(valid_edges)

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), TicTacToeState(remaining_moves={1, 3, 7, 9}), NOT(TicTacToeState(remaining_moves={2, 4, 6, 8})))
    def prioritize_diagonals(self):
        diagonals = [1, 3, 7, 9]
        valid_diagonals = [diagonal for diagonal in diagonals if self.is_valid(diagonal)]
        if valid_diagonals:
            self.computer_move = random.choice(valid_diagonals)

#import msvcrt

def play_game():
    game = TicTacToe()
    print("Game Start")
    game.display_board_initial()
    # while not game.game_over:
    #     print("Human's Turn :")
    #     game.display_board()
    #     human_move = input("Enter your move (1-9): ")
        
    #     game.make_move("X", int(human_move))
    #     print("Human chose position", human_move)




    while not game.game_over:
        print("Human's Turn :")
        game.display_board()
        human_move = input("Enter your move (1-9): ")


        if human_move.isdigit() and int(human_move) in game.remaining_moves:
            if game.make_move("X", int(human_move)):
                print("Human chose position", human_move)
            else:
                continue  # Prompt user to enter move again if it's invalid
        
        # elif human_move == b'\x1b':  # Escape key
        #     break

        else:
            print("Invalid move! Please enter a valid position (1-9) that is not already taken.")
            continue  # Prompt user to enter move again if it's invalid

        if game.remaining_moves and not game.game_over:
            computer_player = TicTacToePlayer(game.board)
            computer_player.reset()
            computer_player.declare(TicTacToeState(remaining_moves=game.remaining_moves))
            computer_player.get_computer_move(game.remaining_moves)
            computer_player.run()

            if computer_player.computer_move:
                position = computer_player.computer_move
                print(f"Computer chose position {position}.")
                game.make_move("O", position)
                if game.game_over:
                    game.display_board()
                    print("Computer wins!")
            else:
                print("No move made by the computer!")

        if not game.game_over and not game.remaining_moves:
            print("It's a tie!")
            break

    game.display_board()

if __name__ == "__main__":
    play_game()


Game Start
 1   2   3  

 4   5   6  

 7   8   9  

Human's Turn :
   |   |   
---|---|---
   |   |   
---|---|---
   |   |   
---|---|---
Human chose position 5
Computer chose position 8.
Human's Turn :
   |   |   
---|---|---
   | X |   
---|---|---
   | O |   
---|---|---
Human chose position 7
Computer chose position 4.
Human's Turn :
   |   |   
---|---|---
 O | X |   
---|---|---
 X | O |   
---|---|---
Player X wins!
Human chose position 3
   |   | X 
---|---|---
 O | X |   
---|---|---
 X | O |   
---|---|---


## GUI test


In [10]:
from pyknow import KnowledgeEngine, Rule, MATCH, Fact
import random
import tkinter as tk
from tkinter import messagebox

class TicTacToeState(Fact):
    pass

class TicTacToe(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def reset(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.remaining_moves = set(range(1, 10))
        self.game_over = False

    def make_move(self, player, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        if self.board[row][col] == " ":
            self.board[row][col] = player
            self.remaining_moves.remove(position)
            if self.check_win(player):
                self.game_over = True
                return True
        else:
            return False  # Indicate that the move was invalid
        return True  # Indicate that the move was valid

    def check_win(self, player):
        # Check rows
        for row in self.board:
            if all(cell == player for cell in row):
                return True
        # Check columns
        for col in range(3):
            if all(self.board[row][col] == player for row in range(3)):
                return True
        # Check diagonals
        if all(self.board[i][i] == player for i in range(3)) or all(self.board[i][2 - i] == player for i in range(3)):
            return True
        return False

class TicTacToePlayer(KnowledgeEngine):
    def __init__(self, board):
        super().__init__()
        self.board = board
        self.computer_move = None  # Store the computer's move

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves))
    def get_computer_move(self, remaining_moves):
        valid_moves = [move for move in remaining_moves if self.is_valid(move)]
        if valid_moves:
            position = random.choice(valid_moves)
            self.computer_move = position  # Store the selected position

    def is_valid(self, position):
        row, col = (position - 1) // 3, (position - 1) % 3
        return self.board[row][col] == " "

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={2, 4, 6, 8})))
    def prioritize_corner(self):
        if self.is_valid(5):
            self.computer_move = 5
        else:
            corners = [1, 3, 7, 9]
            valid_corners = [corner for corner in corners if self.is_valid(corner)]
            if valid_corners:
                self.computer_move = random.choice(valid_corners)

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={5})))
    def choose_center(self):
        if self.is_valid(5):
            self.computer_move = 5

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), NOT(TicTacToeState(remaining_moves={1, 3, 7, 9})))
    def prioritize_edges(self):
        edges = [2, 4, 6, 8]
        valid_edges = [edge for edge in edges if self.is_valid(edge)]
        if valid_edges:
            self.computer_move = random.choice(valid_edges)

    @Rule(TicTacToeState(remaining_moves=MATCH.remaining_moves), TicTacToeState(remaining_moves={1, 3, 7, 9}), NOT(TicTacToeState(remaining_moves={2, 4, 6, 8})))
    def prioritize_diagonals(self):
        diagonals = [1, 3, 7, 9]
        valid_diagonals = [diagonal for diagonal in diagonals if self.is_valid(diagonal)]
        if valid_diagonals:
            self.computer_move = random.choice(valid_diagonals)

class TicTacToeGUI:
    def __init__(self, root):
        self.root = root
        self.game = TicTacToe()
        self.buttons = []
        self.create_board()

    def create_board(self):
        for i in range(3):
            row = []
            for j in range(3):
                button = tk.Button(self.root, text=" ", font=('normal', 40), width=5, height=2,
                                    command=lambda row=i, col=j: self.human_move(row, col),
                                    fg="black", activeforeground="black", highlightthickness=2, highlightbackground="black")
                button.grid(row=i, column=j)
                row.append(button)
            self.buttons.append(row)

    def human_move(self, row, col):
        position = row * 3 + col + 1
        if position in self.game.remaining_moves:
            self.game.make_move("X", position)
            self.buttons[row][col].config(text="X", state=tk.DISABLED, fg="red", activeforeground="red")
            if self.game.game_over:
                messagebox.showinfo("Game Over", "You win!")
                self.reset_game()
            elif not self.game.remaining_moves:
                messagebox.showinfo("Game Over", "It's a tie!")
                self.reset_game()
            else:
                self.computer_move()
        else:
            messagebox.showwarning("Invalid Move", "This position is already taken!")

    def computer_move(self):
        computer_player = TicTacToePlayer(self.game.board)
        computer_player.reset()
        computer_player.declare(TicTacToeState(remaining_moves=self.game.remaining_moves))
        computer_player.get_computer_move(self.game.remaining_moves)
        computer_player.run()

        if computer_player.computer_move:
            position = computer_player.computer_move
            row, col = (position - 1) // 3, (position - 1) % 3
            self.game.make_move("O", position)
            self.buttons[row][col].config(text="O", state=tk.DISABLED, fg="blue", activeforeground="blue")
            if self.game.game_over:
                messagebox.showinfo("Game Over", "Computer wins!")
                self.reset_game()
            elif not self.game.remaining_moves:
                messagebox.showinfo("Game Over", "It's a tie!")
                self.reset_game()
        else:
            messagebox.showinfo("Game Over", "It's a tie!")
            self.reset_game()

    def reset_game(self):
        self.game.reset()
        for row in self.buttons:
            for button in row:
                button.config(text=" ", state=tk.NORMAL, fg="black", activeforeground="black")

def main():
    root = tk.Tk()
    root.title("Tic Tac Toe")
    TicTacToeGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
